In [1]:
import os
# Switch to double, for testing correctness
os.environ['UNIT_TESTING'] = 'True'

In [2]:
import tensor_comprehensions as tc
import torch
import tc_composer
from torch.nn import Conv2d
from tc_composer.implementation import AlexNet
from tc_composer.func.affine_transform import FusedAffineTransform

[INFO] tc_composer.settings - Setting default tensor type: torch.cuda.DoubleTensor
[INFO] tc_composer.settings - Setting epsilon: 1e-16
[INFO] tc_composer.settings - Input tensor shape checking: True
[INFO] tc_composer.settings - Saving compiled options in: /home/ubuntu/tc_composer/options
[INFO] tc_composer.settings - Current CUDA device: Tesla V100-SXM2-16GB
[INFO] tc_composer.settings - Listing CUDA devices:


Found 1 CUDA devices
id 0    b'Tesla V100-SXM2-16GB'                              [SUPPORTED]
                      compute capability: 7.0
                           pci device id: 30
                              pci bus id: 0
Summary:
	1/1 devices are supported


# AlexNet
Implementing https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py

In [4]:
num_classes = 1000

In [5]:
alex_net = AlexNet()

In [6]:
batch_size = 1
in_channels, in_height, in_width = 3, 227, 227
tc_inp = tc_image = torch.randn(batch_size, in_channels, in_height, in_width)

In [7]:
print(alex_net.tc_def(tc_inp))

def AlexNet(
    double(1, 3, 227, 227) input,
    double(64, 3, 11, 11) weight,
    double(64) bias,
    double(192, 64, 5, 5) weight1,
    double(192) bias1,
    double(384, 192, 3, 3) weight2,
    double(384) bias2,
    double(256, 384, 3, 3) weight3,
    double(256) bias3,
    double(256, 256, 3, 3) weight4,
    double(256) bias4,
    double(4096, 9216) weight5,
    double(4096) bias5,
    double(4096, 4096) weight6,
    double(4096) bias6,
    double(1000, 4096) weight7,
    double(1000) bias7
) -> (
    double(1, 64, 56, 56) output,
    double(1, 64, 27, 27) output1,
    double(1, 192, 27, 27) output2,
    double(1, 192, 13, 13) output3,
    double(1, 384, 13, 13) output4,
    double(1, 256, 13, 13) output5,
    double(1, 256, 13, 13) output6,
    double(1, 256, 6, 6) output7,
    double(1, 4096) output8,
    double(1, 4096) output9,
    double(1, 1000) output10
)
{
    output(n, m, h, w) +=! input(n, c, max(min(4*h + kh - 2, 226), 0), max(min(4*w + kw - 2, 226), 0)) * weight(m, 

# Correctness

In [8]:
from torchvision.models import AlexNet

torch_alex_net = AlexNet()
torch_alex_net.train(False)  # Disable dropout

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [9]:
for p, p1 in zip(torch_alex_net.parameters(), alex_net.params):
    p.data = p1.data

torch.Size([64, 3, 11, 11])
torch.Size([64, 3, 11, 11])
torch.Size([64])
torch.Size([64])
torch.Size([192, 64, 5, 5])
torch.Size([192, 64, 5, 5])
torch.Size([192])
torch.Size([192])
torch.Size([384, 192, 3, 3])
torch.Size([384, 192, 3, 3])
torch.Size([384])
torch.Size([384])
torch.Size([256, 384, 3, 3])
torch.Size([256, 384, 3, 3])
torch.Size([256])
torch.Size([256])
torch.Size([256, 256, 3, 3])
torch.Size([256, 256, 3, 3])
torch.Size([256])
torch.Size([256])
torch.Size([4096, 9216])
torch.Size([4096, 9216])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096, 4096])
torch.Size([4096, 4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([1000, 4096])
torch.Size([1000, 4096])
torch.Size([1000])
torch.Size([1000])


In [10]:


start_option = tc.MappingOptions('naive')\
    .mapToBlocks([1000, 6, 512])\
    .mapToThreads([6, 6])\
    .tile([16384, 48, 1024, 1, 4096])\
    .intraTileScheduleFusionStrategy('Max')\
    .outerScheduleFusionStrategy('Max')\
    .unroll(32)\
    .fixParametersBeforeScheduling(False)\
    .matchLibraryCalls(True)

In [11]:
alex_net.recompile(tc_inp, option=start_option)

[INFO] AlexNet - Compiling for input shape - [(1, 3, 227, 227)].


In [ ]:
one = alex_net(tc_inp).data
two = torch_alex_net.forward(tc_inp).data

In [15]:
import numpy as np

np.testing.assert_allclose(one, two, rtol=1e-10)